When Python executes a script and encounters a situation that it cannot cope with, it:

- stops your program;
- creates a special kind of data, called an exception. Of course, this exception is an object.

Both of these activities are called raising an exception. We can say that Python always raises an exception (or that an exception has been raised) when it has no idea what do to with your code.



In [4]:
try:
    print(int('a'))
except ValueError:
    print('You tried to do a nasty thing...')

try:
    print(int('a'))
except ValueError as e_variable:
    print(e_variable.args)

try:
    import abcdefghijk
except ImportError as e:
    # Some exception objects carry additional information about the exception itself.
    print(e.args)
    print(e.name)
    print(e.path)

You tried to do a nasty thing...
("invalid literal for int() with base 10: 'a'",)
("No module named 'abcdefghijk'",)
abcdefghijk
None


In [5]:
try:
    b'\x80'.decode("utf-8")
except UnicodeError as e: 
    # It is a subclass of ValueError
    print(e)
    print(e.encoding) # the name of the encoding that raised the error.
    print(e.reason) # a string describing the specific codec error
    print(e.object) # the object the codec was attempting to encode or decode
    print(e.start) # the first index of invalid data in the object
    print(e.end) # the index after the last invalid data in the object

'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte
utf-8
invalid start byte
b'\x80'
0
1


**Exception chaining**

This chaining concept introduces two attributes of exception instances:

- the __context__ attribute, which is inherent for implicitly chained exceptions;
- the __cause__ attribute, which is inherent for explicitly chained exceptions.

Those attributes help the programmer to keep a reference to the original exception object in a handy and consistent way for later processing like logging, etc

In [8]:
a_list = ['First error', 'Second error']

try:
    print(a_list[3])
except Exception as e:
    print(0 / 0)


ZeroDivisionError: division by zero

In [9]:
a_list = ['First error', 'Second error']

try:
    print(a_list[3])
except Exception as e:
    try:
        # the following line is a developer mistake - they wanted to print progress as 1/10	but wrote 1/0
        print(1 / 0)
    except ZeroDivisionError as f:
        print('Inner exception (f):', f)
        print('Outer exception (e):', e)
        print('Outer exception referenced:', f.__context__)
        print('Is it the same object:', f.__context__ is e)


Inner exception (f): division by zero
Outer exception (e): list index out of range
Outer exception referenced: list index out of range
Is it the same object: True


In [10]:
class RocketNotReadyError(Exception):
    pass


def personnel_check():
    try:
        print("\tThe captain's name is", crew[0])
        print("\tThe pilot's name is", crew[1])
        print("\tThe mechanic's name is", crew[2])
        print("\tThe navigator's name is", crew[3])
    except IndexError as e:
        raise RocketNotReadyError('Crew is incomplete') from e

crew = ['John', 'Mary', 'Mike']
print('Final check procedure')

personnel_check()


Final check procedure
	The captain's name is John
	The pilot's name is Mary
	The mechanic's name is Mike


RocketNotReadyError: Crew is incomplete

In [11]:
class RocketNotReadyError(Exception):
    pass


def personnel_check():
    try:
        print("\tThe captain's name is", crew[0])
        print("\tThe pilot's name is", crew[1])
        print("\tThe mechanic's name is", crew[2])
        print("\tThe navigator's name is", crew[3])
    except IndexError as e:
        raise RocketNotReadyError('Crew is incomplete') from e

crew = ['John', 'Mary', 'Mike']
print('Final check procedure')

try:
    personnel_check()
except RocketNotReadyError as f:
    print('General exception: "{}", caused by "{}"'.format(f, f.__cause__))


Final check procedure
	The captain's name is John
	The pilot's name is Mary
	The mechanic's name is Mike
General exception: "Crew is incomplete", caused by "list index out of range"


In [12]:
# FROM AN EXTENDED CHECKLIST
class RocketNotReadyError(Exception):
    pass


def personnel_check():
    try:
        print("\tThe captain's name is", crew[0])
        print("\tThe pilot's name is", crew[1])
        print("\tThe mechanic's name is", crew[2])
        print("\tThe navigator's name is", crew[3])
    except IndexError as e:
        raise RocketNotReadyError('Crew is incomplete') from e


def fuel_check():
    try:
        print('Fuel tank is full in {}%'.format(100 / 0))
    except ZeroDivisionError as e:
        raise RocketNotReadyError('Problem with fuel gauge') from e


crew = ['John', 'Mary', 'Mike']
fuel = 100
check_list = [personnel_check, fuel_check]

print('Final check procedure')

for check in check_list:
    try:
        check()
    except RocketNotReadyError as f:
        print('RocketNotReady exception: "{}", caused by "{}"'.format(f, f.__cause__))


Final check procedure
	The captain's name is John
	The pilot's name is Mary
	The mechanic's name is Mike
RocketNotReady exception: "Crew is incomplete", caused by "list index out of range"
RocketNotReady exception: "Problem with fuel gauge", caused by "division by zero"


**Scenario**

- Try to extend the check list script to handle more different checks, all reported as RocketNotReady exceptions.
- Add your own checks for: batteries and circuits.


In [15]:
class RocketNotReadyError(Exception):
    pass


def personnel_check():
    try:
        print("\tThe captain's name is", crew[0])
        print("\tThe pilot's name is", crew[1])
        print("\tThe mechanic's name is", crew[2])
        print("\tThe navigator's name is", crew[3])
    except IndexError as e:
        raise RocketNotReadyError('Crew is incomplete') from e


def fuel_check():
    try:
        print('Fuel tank is full in {}%'.format(100/0))
    except ZeroDivisionError as e:
        raise RocketNotReadyError('Problem with fuel gauge') from e

def batteries_check():
    # add your own implementation
    try:
        print(f"The battery level is {int("80%")}")
    except ValueError as e:
        raise RocketNotReadyError('The percentage of the battery is not available to be read') from e

def circuits_check():
    # add your own implementation
    try:
        is_connected = bool(connection)
    except NameError as e:
        raise RocketNotReadyError("The connection is not established") from e


crew = ['John', 'Mary', 'Mike']
fuel = 100
check_list = [personnel_check, fuel_check, batteries_check, circuits_check]

print('Final check procedure')

for check in check_list:
    try:
        check()
    except RocketNotReadyError as f:
        print('RocketNotReady exception: "{}", caused by "{}"'.format(f, f.__cause__))


Final check procedure
	The captain's name is John
	The pilot's name is Mary
	The mechanic's name is Mike
RocketNotReady exception: "Crew is incomplete", caused by "list index out of range"
RocketNotReady exception: "Problem with fuel gauge", caused by "division by zero"
RocketNotReady exception: "The percentage of the battery is not available to be read", caused by "invalid literal for int() with base 10: '80%'"
RocketNotReady exception: "The connection is not established", caused by "name 'connection' is not defined"


Python allows us to operate on the traceback details because each exception object (not only chained ones) owns a __traceback__ attribute. 

In [16]:
class RocketNotReadyError(Exception):
    pass


def personnel_check():
    try:
        print("\tThe captain's name is", crew[0])
        print("\tThe pilot's name is", crew[1])
        print("\tThe mechanic's name is", crew[2])
        print("\tThe navigator's name is", crew[3])
    except IndexError as e:
        raise RocketNotReadyError('Crew is incomplete') from e


crew = ['John', 'Mary', 'Mike']

print('Final check procedure')

try:
    personnel_check()
except RocketNotReadyError as f:
    print(f.__traceback__)
    print(type(f.__traceback__))


Final check procedure
	The captain's name is John
	The pilot's name is Mary
	The mechanic's name is Mike
<class 'traceback'>


From the output presented on the previous page, we can conclude that we have to deal with a traceback type object.

To achieve this, we could use the format_tb() method delivered by the built-in traceback module to get a list of strings describing the traceback.

We could use the print_tb() method, also delivered by the traceback module, to print strings directly to the standard output.

The corresponding output reveals the sequence of exceptions and proves that the execution was not interrupted by the exceptions because we see the final wording Final check is over.

In [ ]:
import traceback

class RocketNotReadyError(Exception):
    pass


def personnel_check():
    try:
        print("\tThe captain's name is", crew[0])
        print("\tThe pilot's name is", crew[1])
        print("\tThe mechanic's name is", crew[2])
        print("\tThe navigator's name is", crew[3])
    except IndexError as e:
        raise RocketNotReadyError('Crew is incomplete') from e


crew = ['John', 'Mary', 'Mike']

print('Final check procedure')

try:
    personnel_check()
except RocketNotReadyError as f:
    print(f.__traceback__)
    print(type(f.__traceback__))
    print('\nTraceback details')
    details = traceback.format_tb(f.__traceback__)
    print("\n".join(details))

print('Final check is over')
